In [ ]:
!pip install tensorflow opencv-python-headless matplotlib

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import cv2
import numpy as np
import os

# Load CSV file with bounding box coordinates
csv_path = '/content/drive/MyDrive/AISecApp/non_target_faces2.csv'
df = pd.read_csv(csv_path)

# Load images and bounding boxes
def load_images_and_bboxes(df, image_dir):
    images = []
    bboxes = []
    filenames = df['Frame'].unique()
    for filename in filenames:
        image_path = os.path.join(image_dir, filename)
        image = cv2.imread(image_path)
        if image is not None:
            images.append(image)
            bbox_list = df[df['Frame'] == filename][['X', 'Y', 'Width', 'Height']].values
            bboxes.append(bbox_list)
    return images, bboxes

# Define image directory
image_dir = '/content/drive/MyDrive/AISecApp/detect_test2'
images, bboxes = load_images_and_bboxes(df, image_dir)

In [ ]:
def apply_mosaic(image, bboxes, kernel_size=(10, 10)):
    for bbox in bboxes:
        x, y, w, h = bbox
        roi = image[y:y+h, x:x+w]
        mosaic = cv2.resize(roi, (kernel_size[1], kernel_size[0]), interpolation=cv2.INTER_LINEAR)
        mosaic = cv2.resize(mosaic, (w, h), interpolation=cv2.INTER_NEAREST)
        image[y:y+h, x:x+w] = mosaic
    return image

# Apply mosaic to each image
mosaic_images = []
for image, bbox_list in zip(images, bboxes):
    mosaic_image = apply_mosaic(image.copy(), bbox_list)
    mosaic_images.append(mosaic_image)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the trained model
autoencoder = tf.keras.models.load_model('content/drive/My Drive/AISecApp/autoencoder_mid.keras')

In [ ]:
import matplotlib.pyplot as plt

# Generate predictions
train_images = [cv2.resize(img, (416, 416)) for img in images]
train_images = np.array(train_images).astype('float32') / 255.0
predicted_images = autoencoder.predict(train_images)

# Convert images back to uint8
predicted_images = (predicted_images * 255).astype(np.uint8)

# Save predicted images
output_dir = '/content/drive/My Drive/AISecApp/output_images'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for i, img in enumerate(predicted_images):
    cv2.imwrite(os.path.join(output_dir, f'frame_{i}.jpg'), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

# Integrate frames into a video
video_path = '/content/drive/My Drive/AISecApp/output_video_1.mp4'
frame_rate = 30

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
height, width, _ = predicted_images[0].shape
video_writer = cv2.VideoWriter(video_path, fourcc, frame_rate, (width, height))

for img in predicted_images:
    video_writer.write(cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

video_writer.release()

4/4 [==============================] - 5s 1s/step
